In [36]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import mplfinance as mpf

start = '2016-01-01'
end = dt.datetime.now()
# stock_code = '^GSPC'
stock_code = 'AMD'
df = yf.download(stock_code, start, end, interval='1d')
# print(df.head(65))

# 何日間で最大最小の値を保持
shortTerm = 20
longTerm = 60

# 損切り率と利確の日数
stopLoss = 0.95
rikaku_day = 80

# judge U trend or Down trend by DC
buy_position = False # 1 means entered and 0 means not already entered
sell_position = False

# 買った（売った）後に経過した日数、損切りラインを保持
buy_dates = 0
sell_dates = 0
buy_stopLine = 0
sell_stopLine = 0
counter = 1
percentChange = []
# assets = []

# chek the highest price in the past {term} times
df['Highest'+str(shortTerm)] = df.iloc[:, 4].rolling(window=shortTerm).max()
# chek the highest price in the past {term} times
df['Lowest'+str(shortTerm)] = df.iloc[:, 4].rolling(window=shortTerm).min()

# chek the highest price in the past {term} times
df['Highest'+str(longTerm)] = df.iloc[:, 4].rolling(window=longTerm).max()
# chek the highest price in the past {term} times
df['Lowest'+str(longTerm)] = df.iloc[:, 4].rolling(window=longTerm).min()

# df.iloc[:,1]
# df['Highest'+str(longTerm)][:65]
# df['Lowest'+str(longTerm)][:65]

"""
メインループ
基本戦略
過去longTerm間の調整済み終値の最高値をその日の調整済み終値が更新したら購入or空売り
損切り率は5%。80日後に利確する。
最終日にまだ保持していた場合捌く
"""
# for i in range(1, len(df)):
for i in range(1,len(df)):
#     print(buy_position, sell_position, buy_dates, sell_dates, buy_stopLine, sell_stopLine, 'Adj Close:{}'.format(df['Adj Close'][i]))
#     if counter == 100:
#         break
    
    shortHighest = df['Highest'+str(shortTerm)][i-1]
    shortLowest = df['Lowest'+str(shortTerm)][i-1]
    longHighest = df['Highest'+str(longTerm)][i-1]
    longLowest = df['Lowest'+str(longTerm)][i-1] 
    # high_price = df['High'][i]
    # low_price = df['Low'][i]
    close = df['Adj Close'][i]
    
    # avoid NaN data 
    # 買いトレンド
    if (np.isnan(longHighest)) == False:
        if (close > longHighest and buy_position == False):
            print('Up trend')
            buy_position = True
            buy_price = close
            buy_stopLine = close * stopLoss
            print('Date:'+str(i),'Buy at the price {}'.format(buy_price))
    
    # 売りトレンド
    if (np.isnan(longLowest)) == False:
        if (close < longLowest and sell_position == False):
            print('Down trend')
            sell_position = True
            sell_price = close
            sell_stopLine = close * stopLoss
            print('Date:'+str(i),'Short Sell at the price {}'.format(sell_price))

    # 損切り
    if buy_position == True and close < buy_price*stopLoss:
        buy_position = False
        percent = (close/buy_price - 1) * 100
        percentChange.append(percent)
        buy_dates = 0
        print('Date:'+str(i),'Stop Loss Buy at the price {}'.format(close))
    if sell_position == True and close > (sell_price + sell_price*(1-stopLoss)):
        sell_position = False
        percent = (sell_price/close - 1) * 100
        percentChange.append(percent)
        sell_dates = 0
        print('Date:'+str(i),'Stop Loss Sell at the price {}'.format(close))

    # 利確
    if buy_dates == 80:
        buy_position = False
        percent = (close/buy_price - 1) * 100
        percentChange.append(percent)
        buy_dates = 0
        print('Date:'+str(i),'Secure Profit Buy at the price {}'.format(close))
    if sell_dates == 80:
        sell_position = False
        percent = (sell_price/close - 1) * 100
        percentChange.append(percent)
        sell_dates = 0
        print('Date:'+str(i),'Secure Profit Sell at the price {}'.format(close))
        
    # 損切りラインの更新
    # if buy_position = 

    #  最終日にまだ持ってたら利確する
    if (counter == df['Adj Close'].count() - 1):
        if buy_position == True:
            buy_position = False
            percent = (close / buy_price - 1) * 100
            percentChange.append(percent)
            buy_dates = 0
            print('Date:'+str(i),'Last Day Buy at the price {}'.format(close))
        if sell_position == True:
            sell_position = False
            percent = (sell_price/close - 1) * 100
            percentChange.append(percent)
            sell_dates = 0
            print('Date:'+str(i),'Last Day Sell at the price {}'.format(close))

    # 買ってからの日数を更新する
    if buy_position == True:
        buy_dates += 1
    if sell_position == True:
        sell_dates += 1
    counter += 1



print(percentChange)

# statistic
gains = 0
numGains = 0
losses = 0
numLosses = 0
total_return = 1

for i in percentChange:
    if i > 0:
        numGains += 1
        gains += i
    else:
        numLosses += 1
        losses += i
    total_return = total_return * ((i / 100) + 1)

total_return = round((total_return - 1)*100, 2)

if numGains > 0:
    average_gain = gains / numGains
    max_return = max(percentChange)
else:
    average_gain = 0
    max_return = 'unknown'
    
if numLosses > 0:
    average_loss = losses / numLosses
    max_loss = min(percentChange)
    risk_reward_retio = - average_gain / average_loss
else:
    average_loss = 0
    max_loss = 'unknown'
    risk_reward_retio = 'inf'
    
if numGains > 0 or numLosses > 0:
    batting_ratio = numGains / (numGains + numLosses)
else:
    batting_ratio = 0
    
print('The period is from {} up to {}'.format(df.index[0], df.index[-1]))
print('Trades: {}'.format(numGains+numLosses))
print('Total return: {}%'.format(total_return))
print('Average Gain: {}'.format(average_gain))
print('Average Loss: {}'.format(average_loss))
print('Max Return: {}'.format(max_return))
print('Max Loss: {}'.format(max_loss))
print('Gain/Loss Ratio: {}'.format(risk_reward_retio))
print('Batting Average: {}'.format(batting_ratio))

[*********************100%***********************]  1 of 1 completed
Up trend
Date:77 Buy at the price 3.990000009536743
Date:78 Stop Loss Buy at the price 3.450000047683716
Up trend
Date:98 Buy at the price 4.039999961853027
Date:178 Secure Profit Buy at the price 6.019999980926514
Up trend
Date:223 Buy at the price 8.460000038146973
Date:303 Secure Profit Buy at the price 13.649999618530273
Down trend
Date:335 Short Sell at the price 10.319999694824219
Date:342 Stop Loss Sell at the price 11.069999694824219
Up trend
Date:394 Buy at the price 14.760000228881836
Date:396 Stop Loss Buy at the price 13.949999809265137
Down trend
Date:459 Short Sell at the price 12.010000228881836
Date:517 Stop Loss Sell at the price 12.649999618530273
Up trend
Date:518 Buy at the price 12.9399995803833
Date:527 Stop Loss Buy at the price 11.569999694824219
Down trend
Date:562 Short Sell at the price 10.0
Date:576 Stop Loss Sell at the price 10.520000457763672
Up trend
Date:597 Buy at the price 12.8199996

In [37]:
"""
メインループ
比較対象：最初の日に買って、最後の日に売る
"""

percentChange = []
counter = 1
print(df.index[:5])

for i in range(1,len(df)):
#     print(counter)
#     print(counter,df.index[i])
#     print(buy_position, sell_position, buy_dates, sell_dates, buy_stopLine, sell_stopLine, 'Adj Close:{}'.format(df['Adj Close'][i]))
#     if counter == 100:
#         break
    close = df['Adj Close'][i]
    
    if i == 1:
        buy_price = close
        print('Buy at the price {}'.format(buy_price))
        
    if (counter == df['Adj Close'].count() - 1):
        percent = (close/buy_price - 1) * 100
        percentChange.append(percent)
        print('Sell at the price {}'.format(close))
        
    counter += 1


print(percentChange)

# statistic
gains = 0
numGains = 0
losses = 0
numLosses = 0
total_return = 1

for i in percentChange:
    if i > 0:
        numGains += 1
        gains += i
    else:
        numLosses += 1
        losses += i
    total_return = total_return * ((i / 100) + 1)

total_return = round((total_return - 1)*100, 2)

if numGains > 0:
    average_gain = gains / numGains
    max_return = max(percentChange)
else:
    average_gain = 0
    max_return = 'unknown'
    
if numLosses > 0:
    average_loss = losses / numLosses
    max_loss = min(percentChange)
    risk_reward_retio = - average_gain / average_loss
else:
    average_loss = 0
    max_loss = 'unknown'
    risk_reward_retio = 'inf'
    
if numGains > 0 or numLosses > 0:
    batting_ratio = numGains / (numGains + numLosses)
else:
    batting_ratio = 0
    
print('The period is from {} up to {}'.format(df.index[0], df.index[-1]))
print('Trades: {}'.format(numGains+numLosses))
print('Total return: {}%'.format(total_return))
print('Average Gain: {}'.format(average_gain))
print('Average Loss: {}'.format(average_loss))
print('Max Return: {}'.format(max_return))
print('Max Loss: {}'.format(max_loss))
print('Gain/Loss Ratio: {}'.format(risk_reward_retio))
print('Batting Average: {}'.format(batting_ratio))

DatetimeIndex(['2015-12-31', '2016-01-04', '2016-01-05', '2016-01-06',
               '2016-01-07'],
              dtype='datetime64[ns]', name='Date', freq=None)
Buy at the price 2.7699999809265137
Sell at the price 85.63999938964844
[2991.6967501567797]
The period is from 2015-12-31 00:00:00 up to 2021-01-29 00:00:00
Trades: 1
Total return: 2991.7%
Average Gain: 2991.6967501567797
Average Loss: 0
Max Return: 2991.6967501567797
Max Loss: unknown
Gain/Loss Ratio: inf
Batting Average: 1.0
